
# 📊 Notebook 2: Prognose von Fußballergebnissen

In diesem Notebook untersuchen wir, inwieweit Fußballspiele mithilfe historischer Daten vorhergesagt werden können.

## 🎯 Ziel:
1. **Klassifikation**: Vorhersage des Ergebnisses (Heimsieg, Unentschieden, Auswärtssieg)
2. **Regression**: Vorhersage der genauen Anzahl an Toren (für Heim- und Auswärtsteam)


In [1]:

import pandas as pd
import os

# Datenpfad anpassen
directory = '../../shared-data'

# Alle CSV-Dateien laden
data_frames = []
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        df['League'] = filename.replace('.csv', '')
        data_frames.append(df)

# Alle Daten zusammenführen
combined_df = pd.concat(data_frames, ignore_index=True)
combined_df = combined_df[['League', 'Date', 'HomeTeam', 'AwayTeam', 'HomeGoals', 'AwayGoals']]
combined_df.dropna(inplace=True)
combined_df.head()


,League,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals
0,Bundesliga 2,28/07/93,Chemnitz,Wuppertaler,0.0,0.0
1,Bundesliga 2,28/07/93,CZ Jena,Hertha,1.0,1.0
2,Bundesliga 2,28/07/93,Hansa Rostock,Munich 1860,4.0,0.0
3,Bundesliga 2,28/07/93,Homburg,St Pauli,2.0,1.0
4,Bundesliga 2,28/07/93,Mannheim,F Koln,1.0,1.0


In [2]:
from sklearn.preprocessing import LabelEncoder
# Ergebnis erstellen: Klassifikation
def get_result(row):
    if row['HomeGoals'] > row['AwayGoals']:
        return 'HomeWin'
    elif row['HomeGoals'] < row['AwayGoals']:
        return 'AwayWin'
    else:
        return 'Draw'

combined_df['Result'] = combined_df.apply(get_result, axis=1)

# Label-Encoding für Teams
from sklearn.preprocessing import LabelEncoder

le_home = LabelEncoder()
le_away = LabelEncoder()
combined_df['HomeTeam_enc'] = le_home.fit_transform(combined_df['HomeTeam'])
combined_df['AwayTeam_enc'] = le_away.fit_transform(combined_df['AwayTeam'])


In [3]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Feature- und Zielvariablen
X = combined_df[['HomeTeam_enc', 'AwayTeam_enc']]
y = combined_df['Result']

# Trainings-/Testdaten splitten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Klassifikationsmodell trainieren
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Vorhersagen und Bewertung
y_pred = clf.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[ 5834  3638  8512]
 [ 4360  3715  9519]
 [ 6104  5585 17798]]
              precision    recall  f1-score   support

     AwayWin       0.36      0.32      0.34     17984
        Draw       0.29      0.21      0.24     17594
     HomeWin       0.50      0.60      0.54     29487

    accuracy                           0.42     65065
   macro avg       0.38      0.38      0.38     65065
weighted avg       0.40      0.42      0.41     65065



In [4]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Regression für HomeGoals
y_home = combined_df['HomeGoals']
X_train_home, X_test_home, y_train_home, y_test_home = train_test_split(X, y_home, test_size=0.3, random_state=42)

reg_home = RandomForestRegressor(n_estimators=100, random_state=42)
reg_home.fit(X_train_home, y_train_home)
pred_home = reg_home.predict(X_test_home)
print("MAE Heimtore:", mean_absolute_error(y_test_home, pred_home))

# Regression für AwayGoals
y_away = combined_df['AwayGoals']
X_train_away, X_test_away, y_train_away, y_test_away = train_test_split(X, y_away, test_size=0.3, random_state=42)

reg_away = RandomForestRegressor(n_estimators=100, random_state=42)
reg_away.fit(X_train_away, y_train_away)
pred_away = reg_away.predict(X_test_away)
print("MAE Auswärtstore:", mean_absolute_error(y_test_away, pred_away))


MAE Heimtore: 1.0436418421458713
MAE Auswärtstore: 0.9117582637080689



## 📝 Fazit

- Die **Klassifikation** zeigt, dass das Modell einfache Vorhersagen wie Heimsieg, Unentschieden oder Auswärtssieg treffen kann, basierend nur auf Teamnamen.
- Die **Regression** hat durchschnittliche Abweichungen von ca. 0.8–1.2 Toren.
- Für genauere Vorhersagen müsste man mehr Features (z. B. Formkurve, Spielerstatistiken, Heimvorteil über Saisons) einbauen.

➡️ Dieses Notebook bietet eine einfache, aber funktionierende Grundlage für Spielprognosen.
